In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from skimage.color import gray2rgb
from tqdm import tqdm
import tifffile
from PIL import Image
import json

In [ ]:
adata = sc.read("xenium_outs/merged_processed.h5ad")

In [ ]:
del adata.uns["spatial"]

In [ ]:
raw_dir = "/data/projects/zeba/MY_PROJECTS/240130_Xenium_ANCA_SLE_GBM/raw_data"
sample_folders = [os.path.join(raw_dir, folder) for folder in os.listdir(raw_dir) if "human_ANCA" in folder]

In [ ]:
xenium_folders_0 = []
ids_folders = []
for folder in tqdm(sample_folders):
    tmp = os.listdir(folder)
    for folder_tmp in tmp:
        if "output-XET" in folder_tmp:
            xenium_folders_0.append(os.path.join(folder, folder_tmp))
            ids_folders.append(folder_tmp.split("_")[2])

In [ ]:
ids_folders = [e for e in ids_folders if e in adata.obs.Slide_ID.unique()]

In [ ]:
ids_folders

In [ ]:
xenium_folders = []
for i in ids_folders:
    for f in xenium_folders_0:
        if i in f:
            xenium_folders.append(f)

In [ ]:
for i in range(len(ids_folders)):
    print(ids_folders[i], xenium_folders[i])

In [ ]:
print(len(ids_folders))

In [ ]:
LEVEL = 1
px_size = 0.425

In [ ]:
"spatial" in adata.uns.keys()

In [ ]:
adata.uns["spatial"] = {}

In [ ]:
for i, ID in tqdm(enumerate(ids_folders)):
    print(ID)
    mipurl = os.path.join(xenium_folders[i], "morphology_mip.ome.tif")
    print("Processing DAPI")
    with tifffile.TiffFile(mipurl) as tif:
        image = tif.series[0].levels[LEVEL].asarray()

    resample = scipy.ndimage.zoom(image, px_size)
    resample_invert = np.invert(resample)
    resample_invert = (resample_invert-resample_invert.min())/(resample_invert.max()-resample_invert.min())
    rgb = gray2rgb(resample_invert)
        
    print("Adding DAPI to anndata")

    adata.uns["spatial"][f'{ID}'] = {}
    adata.uns["spatial"][f'{ID}']["images"] = {}
    adata.uns["spatial"][f'{ID}']["scalefactors"] = {}
        
    adata.uns["spatial"][f'{ID}']["images"][f"dapi_LEVEL{LEVEL}"] = rgb
    adata.uns["spatial"][f'{ID}']["scalefactors"][f"tissue_dapi_LEVEL{LEVEL}_scalef"] = 1
    adata.uns["spatial"][f'{ID}']["scalefactors"]["fiducial_diameter_fullres"] = 40
    adata.uns["spatial"][f'{ID}']["scalefactors"]["spot_diameter_fullres"] = 40

In [ ]:
sc.pl.spatial(adata[adata.obs.Slide_ID==ids_folders[0]], library_id=ids_folders[0], color="CD3D", img_key=f"dapi_LEVEL{LEVEL}")

In [ ]:
sc.pl.spatial(adata[adata.obs.Slide_ID==ids_folders[0]], library_id=ids_folders[0], img_key=f"dapi_LEVEL{LEVEL}")


In [ ]:
adata

In [ ]:
custom_cmap = {
    'podocyte': '#950404FF',
    'mesangial cell': '#C38961FF',
    'PEC': '#9F5630FF',
    'PapE': '#FFB274FF',
    'VSM/P': '#FF7200FF',
    'EC': '#8C37E5FF',
    'PT': '#53505AFF',
    'ATL': '#495564FF',
    'CNT': '#5C616FFF',
    'DCT': '#B5BCCBFF',
    'DTL': '#DCDCE3FF',
    'TAL': '#B8B69EFF',
    'FIB': '#007A99FF',
    'IC': '#00A9CCFF',
    'NEU': '#32E3FFFF',
    'PC': '#99F8FFFF',
    'T': '#648C16FF',
    'NKC/T': '#7EA13EFF',
    'B': '#8BAC54FF',
    'PL': '#A1BA77FF',
    'N': '#FF0000FF',
    'MAC': '#FF0000FF',
    'MDC': '#FF0000FF',
    'ncMON': '#FF0000FF',
    'MAST': '#FF6600FF',
    'cDC': '#FF9900FF',
    'pDC': '#FFCC00FF',
    'cycMNP': '#FFCC00FF',
    'cycNKC/T': '#FFFF00FF'
}

cell_types_order = ['podocyte', 'PEC', 'mesangial cell', 'PapE', 'VSM/P','EC' ,
                        'PT', 'ATL', 'CNT', 'DCT', 'DTL', 'TAL',
                       'FIB', 'IC', 'NEU', 'PC',
                        'T', 'NKC/T','B', 'PL',
                       'N', 'MAC', 'MDC', 'ncMON', 'MAST',
                       'cDC', 'pDC', 'cycMNP', 'cycNKC/T']

In [ ]:
adata.obs["celltype_l1"] = adata.obs["celltype_l1"].cat.reorder_categories(cell_types_order)

In [ ]:
colors = []
for cat in adata.obs["celltype_l1"].cat.categories:
    colors.append(custom_cmap[cat])
adata.uns["celltype_l1_colors"] = colors

In [ ]:
sc.pl.umap(adata, color="celltype_l1")

In [ ]:
adata.write("xenium_outs/merged_processed.h5ad")

In [ ]:
with open("colormap_celltypes.json", "w") as f:
    json.dump(custom_cmap, f)

In [ ]:
cell_types_order = ['podocyte', 'PEC', 'mesangial cell', 'PapE', 'VSM/P','EC' ,
                        'PT', 'ATL', 'CNT', 'DCT', 'DTL', 'TAL',
                       'FIB', 'IC', 'NEU', 'PC',
                        'T', 'NKC/T','B', 'PL',
                       'N', 'MAC', 'MDC', 'ncMON', 'MAST',
                       'cDC', 'pDC', 'cycMNP', 'cycNKC/T']
import pickle
with open("order_celltypes.json", "w") as f:
    json.dump(cell_types_order, f)